# Preparação Dos Dados


# **MODELO 1**

In [6]:
import pandas as pd
import numpy as np
import os

file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Função para criar defasagens
def criar_defasagens(data, entradas):
    for i in range(entradas, 0, -1):  # Defasagens de entradas a 1
        data[f'Entrada {i}'] = data['Vazao'].shift(i)
    novas_colunas = [f'Entrada {i}' for i in range(entradas, 0, -1)] + ['Vazao']
    data = data[novas_colunas].copy()
    for i in range(1, 12):  # Defasagens de 1 a saídas
        data[f'Saida {i}'] = data['Vazao'].shift(-i)
    return data

# Criar datasets para 15, 30 e 45 defasagens
data_15 = criar_defasagens(data.copy(), 15)
data_30 = criar_defasagens(data.copy(), 30)
data_45 = criar_defasagens(data.copy(), 45)




In [7]:
# Pasta onde os arquivos serão salvos

output_folder = 'data/modelo_1'

# Criar a pasta se ela não existir
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Caminhos para os arquivos Excel
file_path_15 = os.path.join(output_folder, 'data_15_dias.xlsx')
file_path_30 = os.path.join(output_folder, 'data_30_dias.xlsx')
file_path_45 = os.path.join(output_folder, 'data_45_dias.xlsx')

# Salvar os DataFrames em arquivos Excel na pasta específica
data_15.to_excel(file_path_15, index=False)
data_30.to_excel(file_path_30, index=False)
data_45.to_excel(file_path_45, index=False)


# **MODELO 2**



In [4]:
import pandas as pd
import numpy as np
import sys

# Carregar o arquivo Excel
file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Adicionar colunas V1 e V2
data['V1'] = np.nan
data['V2'] = np.nan

# Calcular V1 baseado na coluna Vazao
data.loc[1:, 'V1'] = ((data['Vazao'].iloc[1:] - data['Vazao'].shift(1).iloc[1:]) / data['Vazao'].shift(1).iloc[1:]) * 100

# Calcular V2 baseado na coluna V1
data.loc[1:, 'V2'] = data['V1'].iloc[1:] - data['V1'].shift(1).iloc[1:]


data = data[['V2', 'V1', 'Vazao']]

data


,V2,V1,Vazao
0,NaN,NaN,2110
1,NaN,-0.995261,2089
2,-0.871661,-1.866922,2050
3,-0.084298,-1.951220,2010
4,1.005946,-0.945274,1991
...,...,...,...
17284,-1.364289,0.389105,5418
17285,-3.028455,-2.639350,5275
17286,-1.531266,-4.170616,5055
17287,-0.517811,-4.688427,4818


In [ ]:
# Define os intervalos de defasagem a serem testados
intervalos = [15, 30, 45]

# Define os intervalos de defasagem a serem testados
coluna = input("Digite a coluna desejada para fazer a defasagem (V1, V2 ou Vazao)").capitalize()

if(coluna not in data.columns):
    print("Coluna inexistente")
    sys.exit()

# Loop sobre cada intervalo de defasagem
for intervalo in intervalos:
    for i in range(intervalo, 0, -1):
        data[f'Entrada {i}'] = data[coluna].shift(i)

    # Define a ordem das novas colunas (defasagens de entrada + coluna original 'Vazao')
    novas_colunas = [f'Entrada {i}' for i in range(intervalo, 0, -1)] + [coluna]
    data = data[novas_colunas]

    # Criação das colunas de saída defasadas
    for i in range(1, 12):
        data[f'Saida {i}'] = data[coluna].shift(-i)

    # escolhi  exibir a primeira e a última coluna
    subset_columns =  [data.columns[0], coluna, data.columns[-1]]
    
    print(f"\nDataframe para {intervalo} entradas:")
    print(data.loc[:, subset_columns]) # Exibe as primeiras 2 linhas do DataFrame para verificar o resultado  